In [1]:
def getNameForWiki(name):
    name = name.replace(' ','_')
    name = name.replace('&','%26')
    name = name.replace("'","%27")
    name = name.replace(".","")
    
    return name

In [2]:
def getDataFromWiki(origName, soup):
    valList = []
    
    columns=('name', 'Fate','Founded','HeadQuarters','Type','TotalEmployees','CurStatus', 'Defunct')
    
    fate = None
    founded = None
    headquarters = None
    comType = None
    totalEmployees = None
    curStatus = None
    drFunct = None
    
    cType = soup.find(class_='infobox vcard').find_all('tr')

    for items in cType:
        try:
            curTitle = (items.find('th').text).strip()
            if curTitle == 'Fate':
                fate = items.find('td').text
            if  curTitle == 'Founded':
                founded = items.find('td').text
            if  curTitle == 'Headquarters':
                headquarters = items.find('td').text
            if  curTitle == 'Type':
                comType = items.find('td').text
            if  (curTitle == 'Employees' or curTitle == 'Number of employees'):
                totalEmployees = items.find('td').text
            if  curTitle == 'Current Status':
                curStatus = items.find('td').text
            if  curTitle == 'Defunct':
                drFunct = items.find('td').text

        except:
            pass
    
    valList.append(origName)
    valList.append(fate)
    valList.append(founded)
    valList.append(headquarters)
    valList.append(comType)
    valList.append(totalEmployees)
    valList.append(curStatus)
    valList.append(drFunct)
    
    return(pd.DataFrame([valList], columns=columns))



In [ ]:
mainURL = 'https://en.wikipedia.org/wiki/'

headers = {
    'user-Agent': 'Mozilla/5.0'
    
}

statusList = []
wikiVals = pd.DataFrame(columns=('name', 'Fate','Founded','HeadQuarters','Type','TotalEmployees','CurStatus', 'Defunct'))
data_dfs = []
count = 0 

for origName in companyNameList:
    name = getNameForWiki(origName)
    
    url = mainURL + name
    response = requests.get(url, headers = headers)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    #print(name + '  -- '  + str(response.status_code))
    if response.status_code < 300:
        statusList.append([origName,1])
        try:
            temp_df = getDataFromWiki(origName,soup)
            data_dfs.append(temp_df)
        except:
            statusList.append([origName,0])
    else:
        statusList.append([origName,0])
    
    
    if count > 1 and count%100 == 0:
        wikiVals = pd.concat(data_dfs)
        wikiVals.to_csv('/Users/deepakgautam/octo-parakeet/wikiTable.csv', encoding='utf-8')
        
        
    if count > 1  and count%10 == 0:
        with open("/Users/deepakgautam/octo-parakeet/wikiProgress.txt", "a") as myfile:
            myfile.write("Scrapping complete for " + str(count + 1) + " movies. \n")
            
    count += 1
    
wikiVals = pd.concat(data_dfs)

    

    



